## Compulsory Task 

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [1]:
# Libraries
import pandas as pd
import numpy as np
import fuzzywuzzy
from fuzzywuzzy import process
import chardet
import string
from datetime import datetime

# Set seed for reproducibility
np.random.seed(0)

# Print confirmation
print("Libraries successfully imported.")

Libraries successfully imported.


In [2]:
# Load up store_income_data.csv
store_income_df = pd.read_csv("store_income_data_task.csv")
store_income_df.head()

,id,store_name,store_email,department,income,date_measured,country
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,United States/
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,Britain
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,United States
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,Britain/
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,United Kingdom


1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [3]:
# Display unique values in the 'country' column before cleaning
countries_before_cleaning = store_income_df['country'].unique()
print(f"There are {len(countries_before_cleaning)} unique countries before cleaning:\n")
print(countries_before_cleaning)
print()

There are 77 unique countries before cleaning:

['United States/' 'Britain' ' United States' 'Britain/' ' United Kingdom'
 'U.K.' 'SA ' 'U.K/' 'America' 'United Kingdom' nan 'united states'
 ' S.A.' 'England ' 'UK' 'S.A./' 'ENGLAND' 'BRITAIN' 'U.K' 'U.K '
 'America/' 'SA.' 'S.A. ' 'u.k' 'uk' ' ' 'UK.' 'England/' 'england'
 ' Britain' 'united states of america' 'UK/' 'SA/' 'SA' 'England.'
 'UNITED KINGDOM' 'America.' 'S.A..' 's.a.' ' U.K'
 ' United States of America' 'Britain ' 'England' ' SA'
 'United States of America.' 'United States of America/' 'United States.'
 's. africasouth africa' ' England' 'United Kingdom '
 'United States of America ' ' UK' 'united kingdom' 'AMERICA' 'America '
 'UNITED STATES OF AMERICA' ' S. AfricaSouth Africa' 'america'
 'S. AFRICASOUTH AFRICA' 'Britain.' '/' 'United Kingdom.' 'United States'
 ' America' 'UNITED STATES' 'sa' 'United States of America' 'UK '
 'United States ' 'S. AfricaSouth Africa/' 'S.A.' 'United Kingdom/'
 'S. AfricaSouth Africa ' 'S. 

In [4]:
# Step 1: Convert the 'country' column to lowercase and remove any trailing white spaces
store_income_df['country'] = store_income_df['country'].str.lower().str.strip()

# Display unique values after converting to lowercase and stripping whitespace
countries_after_lower_strip = store_income_df['country'].unique()
print(f"There are {len(countries_after_lower_strip)} unique countries after converting to lowercase and stripping whitespace:\n")
print(countries_after_lower_strip)
print()

There are 37 unique countries after converting to lowercase and stripping whitespace:

['united states/' 'britain' 'united states' 'britain/' 'united kingdom'
 'u.k.' 'sa' 'u.k/' 'america' nan 's.a.' 'england' 'uk' 's.a./' 'u.k'
 'america/' 'sa.' '' 'uk.' 'england/' 'united states of america' 'uk/'
 'sa/' 'england.' 'america.' 's.a..' 'united states of america.'
 'united states of america/' 'united states.' 's. africasouth africa'
 'britain.' '/' 'united kingdom.' 's. africasouth africa/'
 'united kingdom/' 's. africasouth africa.' '.']



In [5]:
# Step 2: Remove punctuation
store_income_df['country'] = store_income_df['country'].str.replace(f"[{string.punctuation}]", "", regex=True)

# Display unique values after removing punctuation
countries_after_punctuation_removal = store_income_df['country'].unique()
print(f"There are {len(countries_after_punctuation_removal)} unique countries after removing punctuation:\n")
print(countries_after_punctuation_removal)
print()

There are 11 unique countries after removing punctuation:

['united states' 'britain' 'united kingdom' 'uk' 'sa' 'america' nan
 'england' '' 'united states of america' 's africasouth africa']



In [6]:
# Step 3: Replace different missing values with pd.NA
store_income_df['country'] = store_income_df['country'].replace(['', 'nan', 'none', None, ' '], pd.NA)

# Display unique values after replacing missing values
countries_after_replacing_missing = store_income_df['country'].unique()
print(f"There are {len(countries_after_replacing_missing)} unique countries after replacing missing values:\n")
print(countries_after_replacing_missing)
print()

There are 10 unique countries after replacing missing values:

['united states' 'britain' 'united kingdom' 'uk' 'sa' 'america' <NA>
 'england' 'united states of america' 's africasouth africa']



In [7]:
# Check rows with empty strings or whitespace-only strings
# If there are any empty rows,
# We would need to remove them
empty_rows = store_income_df[store_income_df['country'].isin(['', ' '])]
print("Rows with empty strings or whitespace-only strings in the 'country' column:\n")
print(empty_rows)

Rows with empty strings or whitespace-only strings in the 'country' column:

Empty DataFrame
Columns: [id, store_name, store_email, department, income, date_measured, country]
Index: []


2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [8]:
# List of standard country names
standard_countries = ['South Africa', 'United Kingdom', 'United States']

# Function to match country names to standard names
def match_country(country, choices):
    if pd.isna(country):
        return country
    match, score = process.extractOne(country, choices)
    if score >= 80:  # You can adjust the threshold for matching
        return match
    return country

# Apply the matching function to the 'country' column
store_income_df['country'] = store_income_df['country'].apply(lambda x: match_country(x, standard_countries))

# Display unique values in the 'country' column after fuzzy matching
countries_after_fuzzy_matching = store_income_df['country'].unique()
print(f"There are {len(countries_after_fuzzy_matching)} unique countries after fuzzy matching:")
print(countries_after_fuzzy_matching)
print()

There are 9 unique countries after fuzzy matching:
['United States' 'britain' 'United Kingdom' 'uk' 'sa' 'america' <NA>
 'england' 'South Africa']



In [9]:
# Mapping of variations to the correct country names
country_mapping = {
    'britain': 'United Kingdom',
    'uk': 'United Kingdom',
    'sa': 'South Africa',
    'america': 'United States',
    'england': 'United Kingdom',
}

# Apply the mapping to the 'country' column
store_income_df['country'] = store_income_df['country'].map(country_mapping).fillna(store_income_df['country'])

# Display unique values in the 'country' column after mapping
countries_after_mapping = store_income_df['country'].unique()
print(f"There are {len(countries_after_mapping)} unique countries after mapping:")
print(countries_after_mapping)
print()

There are 4 unique countries after mapping:
['United States' 'United Kingdom' 'South Africa' <NA>]



In [10]:
# Display unique values in the 'country' column after mapping, ignoring NaN
countries_after_mapping = store_income_df['country'].dropna().unique()
print(f"There are {len(countries_after_mapping)} unique countries after mapping (ignoring NaN):\n")
print(countries_after_mapping)
print()

There are 3 unique countries after mapping (ignoring NaN):

['United States' 'United Kingdom' 'South Africa']



3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [11]:
# Convert 'date_measured' to datetime format
store_income_df['date_measured'] = pd.to_datetime(store_income_df['date_measured'], errors='coerce', dayfirst=True)

# Get the current date
current_date = pd.to_datetime(datetime.today())

# Calculate the number of days ago
store_income_df['days_ago'] = (current_date - store_income_df['date_measured']).dt.days

# Display the DataFrame with the new 'days_ago' column
store_income_df.head()

,id,store_name,store_email,department,income,date_measured,country,days_ago
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,2006-02-04,United States,6739
1,2,Nordson Corporation,NaN,Tools,$41744177.01,2006-01-04,United Kingdom,6770
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,2003-09-12,United States,7615
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,2006-05-08,United Kingdom,6646
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,1973-01-21,United Kingdom,18806
